In [2]:
import torch
import torchvision
from torch import nn
from torchvision import transforms, datasets
from torchinfo import summary

import numpy as np
import matplotlib.pyplot as plt

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
import os
if not os.path.exists("data/10_whole_foods"):
  !wget https://storage.googleapis.com/food-vision-image-playground/10_whole_foods.zip
  !unzip -q 10_whole_foods.zip
else:
  print("10_whole_foods dir exists, skipping download")

# Setup Dirs
train_dir = "data/10_whole_foods/train"
test_dir = "data/10_whole_foods/test"

from torchvision.datasets import ImageFolder

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

simple_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

train_data = ImageFolder(train_dir, transform=simple_transform)
test_data = ImageFolder(test_dir, transform=simple_transform)


BATCH_SIZE = 32
train_dataloader = torch.utils.data.DataLoader(train_data, 
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               num_workers=8,
                                               pin_memory=True)
test_dataloader = torch.utils.data.DataLoader(test_data, 
                                              batch_size=BATCH_SIZE, 
                                              shuffle=True,
                                              num_workers=8,
                                              pin_memory=True)


10_whole_foods dir exists, skipping download


In [5]:
model = torchvision.models.efficientnet_b0(pretrained=True).to(device)

# Print a summary
# summary(model, input_shape=(1, 3, 224, 224))

In [6]:
model

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [7]:
# Update the classifier
model.classifier = torch.nn.Linear(1280, 10).to(device)

# Freeze all base layers (the feature extractor in VGG)
for param in model.features.parameters():
  param.requires_grad = False

# Print a summary
# model

## Make a small train function

In [8]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
from tqdm import tqdm
epochs = 5
for epoch in tqdm(range(epochs)):
  total_loss, acc, count = 0, 0, 0
  for batch, (X, y) in enumerate(train_dataloader):
    model.train()
    X, y = X.to(device), y.to(device)
    
    # 1. Forward pass
    y_pred = model(X)

    # 2. Calc loss and metrics
    loss = loss_fn(y_pred, y)
    total_loss += loss
    y_pred_class = torch.softmax(y_pred, dim=1).argmax(dim=1)
    # print(f"Truth labels----------:\n {y}")
    # print(f"Pred labels----------:\n {y_pred_class}")
    # print(f"Sum: {torch.eq(y_pred, y).sum().item()}")
    acc += torch.eq(y_pred_class, y).sum().item()
    count += len(X)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

  # Test
  test_loss_total, test_acc, test_count = 0, 0, 0
  for batch_test, (X_test, y_test) in enumerate(test_dataloader):
    model.eval() # turn on eval mode
    X_test, y_test = X_test.to(device), y_test.to(device) # put test data to GPU
    with torch.inference_mode(): # turn on inference mode
      test_pred = model(X_test)
      test_loss = loss_fn(test_pred, y_test)
      test_loss_total += loss
      test_pred_class = torch.softmax(test_pred, dim=1).argmax(dim=1)
      # print(f"Test Truth labels----------:\n {y_test}")
      # print(f"Test Pred labels----------:\n {test_pred_class}")
      # print(test_pred_class)
      test_acc += (test_pred_class == y_test).sum()
      test_count += len(X_test)
  
  # Print out what's happening
  print(f""" 
  Train count: {count}
  Test count: {test_count}
  Epoch: {epoch} 
  Train Loss: {total_loss/count:.4f} | Train Acc: {acc/count:.4f}
  Test loss: {test_loss_total/test_count:.4f} | Test Acc: {test_acc/test_count:.4f}
  """)

 20%|██        | 1/5 [00:21<01:27, 21.87s/it]

 
  Train count: 654
  Test count: 164
  Epoch: 0 
  Train Loss: 0.0198 | Train Acc: 0.8869
  Test loss: 0.0168 | Test Acc: 0.8659
  


 40%|████      | 2/5 [00:42<01:03, 21.16s/it]

 
  Train count: 654
  Test count: 164
  Epoch: 1 
  Train Loss: 0.0170 | Train Acc: 0.9128
  Test loss: 0.0289 | Test Acc: 0.8476
  


 60%|██████    | 3/5 [01:02<00:41, 20.63s/it]

 
  Train count: 654
  Test count: 164
  Epoch: 2 
  Train Loss: 0.0141 | Train Acc: 0.9251
  Test loss: 0.0146 | Test Acc: 0.8659
  


 80%|████████  | 4/5 [01:23<00:20, 20.58s/it]

 
  Train count: 654
  Test count: 164
  Epoch: 3 
  Train Loss: 0.0129 | Train Acc: 0.9327
  Test loss: 0.0219 | Test Acc: 0.8598
  


 80%|████████  | 4/5 [01:30<00:22, 22.60s/it]


KeyboardInterrupt: 

In [ ]:
len(train_dataloader) * 32

672